# Web Scrapping

In [1]:
#IMPORTS
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [ ]:
#The following code cell, is an all_inclusive function that access the website, retrieve the information needed and stores them into a dataframe

In [ ]:
def webscrape_data (universe, nb_pages):
    #To initialize the webpage number to 1
    page = 1

    #To create an empty list in which I will add all the characters webpage
    all_pages = []


    #While loop to nb_pages which can be found on the website itself
    while page < nb_pages:
        website_page = f"https://www.superherodb.com/dc-comics/{universe}/?page_nr={page}"

        #To ensure the request of each page is granted or denied
        req =  requests.get(website_page)
        if req.status_code == 200:
            pass
        else:
            print('request has been denied, try again')
    
        #To use beautifulsoup on the request
        soup = BeautifulSoup(req.content, 'html.parser')

        #To find all the class needed
        results = soup.findAll('div', class_= 'column col-12')[2]

        #To set a variable to my results
        results = results.select('a[href]')

        #To create a variable with all the characters webpage
        all_pages.extend(results)

        #To iterate through the while loop
        page += 1   

    #To create an empty dataframe for the universe characters
    universe_df = pd.DataFrame()


    #To identify the website
    website = 'https://www.superherodb.com'

    #To create an empty dataframe to store the names
    df_names = pd.DataFrame()

    #To create an empty dataframe to store the powerstats
    df_powerstats = pd.DataFrame()


    #This for loop extracts all names and powerstats from each characters
    #1- Names
    for page in all_pages:

        #To send a request
        req = requests.get(website+page['href'])

        #To use beautifulsoup for readability purposes
        soup = BeautifulSoup(req.content, 'html.parser')

        #To locate the names
        all_names = soup.find('div', class_ = 'columns profile-titles')

        #To find the names
        names = all_names.find(class_ = 'column col-10 col-md-9 col-sm-12').h1.get_text()

        #To add names into df_names
        df_names= df_names.append([names], ignore_index=True)


    #2- Powerstats
        #To find all the stats in the class needed
        all_stats = soup.find('div', class_= 'stat-bar')

        #In this particular website, the stats where located under scripts, the following line simply fetches the location in a text format
        all_stats = all_stats.parent.find_next_sibling('script').get_text()

        #Furthermore, the stats are located in a subcategory: "bars", this step fetches the stats under bars and splits its content and retain the first element
        #Lastly it is turned into a dictionary
        powerstats = json.loads(all_stats.split('\"bars\":')[1].split(",\"shdbclass")[0])

        #To add powerstats into df_powerstats
        df_powerstats = df_powerstats.append(powerstats, ignore_index=True)


    #To merge all dataframes
    #first merge names and powerstats dataframes
    universe_df = pd.merge(df_names, df_powerstats, left_index = True, right_index = True)

    #To change column names
    universe_df.columns = [['Names', 'Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat', 'Tier']]

    return (universe_df)


#In this particular website, the DC and Marvel url are deffined by the codes below
DC  = '70-2'
Marvel = '70-1'

#To export the data into a CSV file
webscrape_data(DC, 92).to_csv('DC_data')
webscrape_data(Marvel, 103).to_csv('Marvel_data')

In [ ]:
#The function be

In [ ]:
def battles(nb_pages):

    #To initialize iterable variables
    page = 1
    counter = 0
    counter2 = 1

    #To initialize lists
    P1 = []
    P2 = []
    win1 = []
    draw = []
    win2 = []

    while page < nb_pages:
        website_page = f"https://www.superherodb.com/filler/90-{page}/"

        #To ensure the request of each page is granted or denied
        req =  requests.get(website_page)
        if req.status_code == 200:
            pass
        else:
            print('request has been denied, try again')

        #To use beautifulsoup for readability purposes
        soup = BeautifulSoup(requests.get(website_page).content)

        #check if page contains info
        #stat results
        battle_results = soup.find("div", class_ = "battle-result")
        if not battle_results:
            #to skip this page as it doesn't contain results
            continue

        #Battle results
        #P1 = 0, P2 = 1
        Person = soup.find("h1", class_ = "h1-battle").get_text().split("vs")

        battle_results = battle_results.find_all("div")
        P1_win = battle_results[0].get_text().split("(")[1].split(")")[0].rstrip("%")

        draws = battle_results[1].get_text().split("(")[1].split(")")[0].rstrip("%")

        P2_win = battle_results[2].get_text().split("(")[1].split(")")[0].rstrip("%")

        if P1_win:
            P1.append(Person[0])
            P2.append(Person[1])
            win1.append(P1_win)
            draw.append(draws)
            win2.append(P2_win)
            counter +=1
        else:
            pass

        if counter >= 1000 or page == 772194:
            df = pd.DataFrame(list(zip(P1,P2,win1,draw,win2)),columns=['Name1', 'Name2', 'Name1_res', 'Draw', 'Name2_res'])
            #do df to csv here
            df.to_csv(f"battle_results_{counter2}.csv", index =False)
            print(f"Reached battle iteration: {page}")
            counter2 += 1
            counter = 0
            P1 = []
            P2 = []
            win1 = []
            draw = []
            win2 = []
            break

    return

#nb of page is 772194 as of 16May2022, users can post new battles
battles(772194)